In [1]:
import pickle, time
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import re
from pathlib import Path
import pandas as pd
from typing import List, Dict
from tqdm.notebook import tqdm   
import numpy as np        

from dgd.environments.drl3env_loader6 import _apply_implicit_or, _compute_hash, _compute_truth_key

from dgd.utils.utils5 import (
    calculate_truth_table_v2,
    generate_one_hot_features_from_adj,
    resize_matrix,
    energy_score,
    check_implicit_OR_existence_v3,
    add_implicit_OR_to_dag_v2,
    exhaustive_cut_enumeration_dag,
    is_fanout_free_standalone,
    generate_subgraph,
    simulate_signal_propagation_binary,
    plot_circuit_layered,
    write_tt_files
)

from dgd.utils.verification import *

Done loading action motifs. There are 15928 unique motifs.


In [2]:
'''
circuits_hex_list = [
"0x000D",
"0x0239",
"0x0304",
"0x040B",
"0x0575",
"0x057A",
"0x0643",
"0x0760",
"0x09AF",
"0x0F42",
"0x1038",
"0x1048",
"0x10C9",
"0x1284",
"0x1323",
"0x13CE",
"0x1714",
"0x1858",
"0x1A60",
"0x1AC6",
"0x1CBF",
"0x1D95",
"0x1FDE",
"0x226B",
"0x22C6",
"0x23A7",
"0x240F",
"0x2A38",
"0x2A56",
"0x2FC7",
"0x3060",
"0x30CE",
"0x32AA",
"0x35C3",
"0x36DC",
"0x3812",
"0x3A17",
"0x3B31",
"0x3B60",
"0x3B68",
"0x409B",
"0x41A2",
"0x41B2",
"0x429B",
"0x4724",
"0x47FD",
"0x48C1",
"0x4A32",
"0x4BF8",
"0x5215",
"0x53AF",
"0x53D7",
"0x599A",
"0x5AAD",
"0x5B30",
"0x5DA9",
"0x5F01",
"0x5FE2",
"0x616A",
"0x648B",
"0x6572",
"0x680A",
"0x6847",
"0x699D",
"0x6F2A",
"0x7096",
"0x70EC",
"0x7176",
"0x822B",
"0x850E",
"0x8F63",
"0x914C",
"0x918A",
"0x93AC",
"0x9591",
"0x96F7",
"0x9917",
"0x9BF5",
"0x9F8A",
"0xA2DA",
"0xA7B2",
"0xA960",
"0xB744",
"0xB8AD",
"0xBC16",
"0xBCA3",
"0xBDF1",
"0xBEE9",
"0xBF36",
"0xC248",
"0xC4B2",
"0xC766",
"0xCB82",
"0xCBD6",
"0xCE97",
"0xD319",
"0xD326",
"0xD477",
"0xD4E4",
"0xD550",
"0xDA80",
"0xDBFA",
"0xE605",
"0xE677",
"0xE93A",
"0xECF1",
"0xEFEB",
"0xF43F",
"0xF4E7",
"0xF5A4",
"0xFC79"]
'''

#generalization
circuits_hex_list = ["0x0239", "0x040B", "0x0575", "0x0643", "0x0760", "0x09AF", "0x0F42", "0x1048",
           "0x10C9", "0x1284", "0x1714", "0x1858", "0x1AC6", "0x22C6", "0x23A7", "0x240F",
           "0x2A38", "0x2A56", "0x2FC7", "0x3060", "0x35C3", "0x3812", "0x3B68", "0x409B",
           "0x41B2", "0x429B", "0x4724", "0x48C1", "0x4A32", "0x4BF8", "0x53AF", "0x53D7",
           "0x5B30", "0x5DA9", "0x5F01", "0x616A", "0x850E", "0x8F63", "0x93AC",
           "0x9591", "0x96F7", "0x9BF5", "0xA2DA", "0xA7B2", "0xA960", "0xB744", "0xB8AD",
           "0xBC16", "0xBCA3", "0xBDF1", "0xBF36", "0xC248", "0xC4B2", "0xC766", "0xCBD6",
           "0xCE97", "0xD326", "0xDA80", "0xE605", "0xF43F", "0xF5A4"]


print("Lenth of circuits_hex_list: ",len(circuits_hex_list))

for circuit_hex in circuits_hex_list:
    
    
    #scratch
    #run_dir = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/manuscript/scratch_training/4in/{circuit_hex}/seed_2"
    
    # fine-tune
    run_dir = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/manuscript/fine_tune/GAT_MLP_with_scalars/4000nn/initial_state_sampling_factor_0/4_inputs/50_000_steps/{circuit_hex}/seed_3"
    
    
    
    run_dir = Path(run_dir)
    
    # skip if run_dir does not exist
    if not run_dir.exists():
        print(f"Skip: run_dir not found → {run_dir}")
        continue   
    
    rx = re.compile(r"shared_registry_([\d_]+)\.pkl$")

    def tag_value(p: Path) -> int:
        """Return the numeric tag, e.g. '4_800' -> 4800."""
        m = rx.fullmatch(p.name)
        if not m:
            raise ValueError(f"Not a registry file: {p}")
        return int(m.group(1).replace("_", ""))

    final_path = run_dir / "final_shared_registry.pkl"

    if final_path.is_file():
        #print("Using final registry:", final_path)
        pkl_path = final_path
    else:
        candidates = list(run_dir.glob("shared_registry_*.pkl"))
        if not candidates:
            print("No final_shared_registry.pkl or shared_registry_*.pkl found in", run_dir)
            pkl_path = None  # or raise if you prefer
        else:
            latest = max(candidates, key=tag_value)
            print("Latest snapshot:", latest, "steps =", tag_value(latest))
            pkl_path = latest

Lenth of circuits_hex_list:  61
